# ADAR2 Analysis: Gold Standard Validation

Deep dive into ADAR2 structure and IP6 binding site.


In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from cryptic_ip.validation import validate_adar2, download_adar2_structures
from cryptic_ip.analysis import ProteinAnalyzer

sns.set_style('whitegrid')


## Download ADAR2 Structures

In [ ]:
structures = download_adar2_structures()
print(f"AlphaFold: {structures['alphafold']}" )
print(f"Crystal: {structures['crystal']}")

## Analyze AlphaFold Structure

In [ ]:
analyzer = ProteinAnalyzer(str(structures['alphafold']))
pockets = analyzer.detect_pockets()
scored = analyzer.score_all_pockets()

print(f'Detected {len(pockets)} pockets')
print(f'\nTop 5 scoring pockets:')
print(scored[['pocket_id', 'composite_score', 'volume', 'sasa', 'basic_residues']].head())

## Visualize Score Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Score distribution
axes[0].hist(scored['composite_score'], bins=20, edgecolor='black')
axes[0].set_xlabel('Composite Score')
axes[0].set_ylabel('Count')
axes[0].set_title('ADAR2 Pocket Score Distribution')
axes[0].axvline(0.7, color='r', linestyle='--', label='Threshold')
axes[0].legend()

# Volume vs Score
axes[1].scatter(scored['volume'], scored['composite_score'], alpha=0.6)
axes[1].set_xlabel('Pocket Volume (ų)')
axes[1].set_ylabel('Composite Score')
axes[1].set_title('Volume vs Score')
axes[1].axhline(0.7, color='r', linestyle='--')
axes[1].axvspan(300, 800, alpha=0.1, color='green')

plt.tight_layout()
plt.show()